# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Import packages
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format
from pyspark.sql.types import StructType, StructField, DoubleType, StringType, IntegerType, DateType, TimestampType

from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from collections import OrderedDict
import psycopg2

# Set defaults
pd.set_option("max_rows", 1000)
pd.set_option("max_columns", 1000)

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

In [2]:
# Build spark session
spark = SparkSession.builder.config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
                    .enableHiveSupport().getOrCreate()

# Read only 1 file of the immigration dataset to keep the size easy to work with
fpath = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
im_df = spark.read.format('com.github.saurfang.sas.spark').load(fpath,forceLowercaseNames=True).limit(1000)
im_df = im_df.toPandas()

# Read remaining datasets
airport_df = pd.read_csv('airport-codes_csv.csv', nrows=1000)
city_df = pd.read_csv('us-cities-demographics.csv', delimiter=';', nrows=1000)

In [3]:
# Evaluate immigration import
im_df.head()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,6.0,2016.0,4.0,692.0,692.0,XXX,20573.0,NaN,None,NaN,37.0,2.0,1.0,None,None,None,T,None,U,None,1979.0,10282016,None,None,None,1.897628e+09,None,B2
1,7.0,2016.0,4.0,254.0,276.0,ATL,20551.0,1.0,AL,NaN,25.0,3.0,1.0,20130811,SEO,None,G,None,Y,None,1991.0,D/S,M,None,None,3.736796e+09,00296,F1
2,15.0,2016.0,4.0,101.0,101.0,WAS,20545.0,1.0,MI,20691.0,55.0,2.0,1.0,20160401,None,None,T,O,None,M,1961.0,09302016,M,None,OS,6.666432e+08,93,B2
3,16.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,28.0,2.0,1.0,20160401,None,None,O,O,None,M,1988.0,09302016,None,None,AA,9.246846e+10,00199,B2
4,17.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,4.0,2.0,1.0,20160401,None,None,O,O,None,M,2012.0,09302016,None,None,AA,9.246846e+10,00199,B2


In [4]:
im_df.describe()

,cicid,i94yr,i94mon,i94cit,i94res,arrdate,i94mode,depdate,i94bir,i94visa,count,biryear,admnum
count,1000.000000,1000.0,1000.0,1000.00000,1000.000000,1000.000000,999.000000,958.000000,1000.00000,1000.000000,1000.0,1000.00000,1.000000e+03
mean,587.550000,2016.0,4.0,104.01100,115.215000,20545.034000,1.007007,20557.148225,39.08800,1.876000,1.0,1976.91200,5.735365e+10
std,336.493741,0.0,0.0,19.22359,68.854758,0.905353,0.083456,17.152368,16.66259,0.350352,0.0,16.66259,1.105955e+10
min,6.000000,2016.0,4.0,101.00000,101.000000,20545.000000,1.000000,20546.000000,0.00000,1.000000,1.0,1932.00000,6.649566e+08
25%,294.750000,2016.0,4.0,103.00000,103.000000,20545.000000,1.000000,20550.000000,27.00000,2.000000,1.0,1965.00000,5.542676e+10
50%,565.500000,2016.0,4.0,103.00000,103.000000,20545.000000,1.000000,20553.000000,39.00000,2.000000,1.0,1977.00000,5.543622e+10
75%,903.250000,2016.0,4.0,104.00000,104.000000,20545.000000,1.000000,20559.000000,51.00000,2.000000,1.0,1989.00000,5.544390e+10
max,1159.000000,2016.0,4.0,692.00000,692.000000,20573.000000,2.000000,20698.000000,84.00000,3.000000,1.0,2016.00000,9.251028e+10


In [5]:
# Evaluate cities import
city_df.head()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402


In [6]:
city_df.describe()

,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,Count
count,1000.000000,9.980000e+02,9.980000e+02,1.000000e+03,996.000000,9.960000e+02,994.000000,1.000000e+03
mean,35.566900,8.953580e+04,9.320766e+04,1.825231e+05,9095.504016,3.414385e+04,2.745392,4.799123e+04
std,4.583388,1.375867e+05,1.422165e+05,2.794854e+05,11577.450298,8.233672e+04,0.433821,1.227271e+05
min,22.900000,2.928100e+04,2.734800e+04,6.321500e+04,416.000000,1.058000e+03,2.000000,9.800000e+01
25%,32.800000,3.950400e+04,4.147500e+04,8.065300e+04,3666.750000,9.202000e+03,2.440000,3.602750e+03
50%,35.300000,5.097750e+04,5.273050e+04,1.037050e+05,5199.000000,1.845100e+04,2.640000,1.371150e+04
75%,38.100000,7.907100e+04,8.479475e+04,1.634480e+05,9368.000000,3.386450e+04,2.970000,5.113000e+04
max,70.500000,1.958998e+06,2.012898e+06,3.971896e+06,109089.000000,1.485425e+06,4.980000,2.177650e+06


In [25]:
# Evaluate airports import
airport_df.head()

,ident,type,name,elevation_ft,iso_country,iso_region,municipality,gps_code,local_code,airline,latitude,longitude
0,00A,heliport,Total Rf Heliport,11.0,US,US-PA,Bensalem,00A,00A,A,-74.93360137939453,40.07080078125
1,00AA,small_airport,Aero B Ranch Airport,3435.0,US,US-KS,Leoti,00AA,00AA,AA,-101.473911,38.704022
2,00AK,small_airport,Lowell Field,450.0,US,US-AK,Anchor Point,00AK,00AK,AK,-151.695999146,59.94919968
3,00AL,small_airport,Epps Airpark,820.0,US,US-AL,Harvest,00AL,00AL,AL,-86.77030181884766,34.86479949951172
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,US,US-AR,Newport,NaN,NaN,NaN,-91.254898,35.6087


In [26]:
airport_df.describe()

,elevation_ft
count,993.000000
mean,1160.388721
std,1450.449105
min,2.000000
25%,280.000000
50%,720.000000
75%,1215.000000
max,10175.000000


### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

## Immigration Data

Steps include:
1. Check null count by column
2. Check duplicated rows
3. Check data types and update as needed
4.  Check column names and standardize as needed (all lowercase with underscores instead of spaces)
5.  Drop any columns that are completely empty
6.  Check extremes, max, min, and make sure numbers make sense (do areas that have high counts in real life also have high counts in this dataset and vice versa, are averages plausible, etc.)

In [7]:
#Check nulls
im_df.isnull().sum()

cicid          0
i94yr          0
i94mon         0
i94cit         0
i94res         0
i94port        0
arrdate        0
i94mode        1
i94addr       28
depdate       42
i94bir         0
i94visa        0
count          0
dtadfile       1
visapost     941
occup       1000
entdepa        0
entdepd       42
entdepu      998
matflag       42
biryear        0
dtaddto        0
gender       190
insnum      1000
airline        2
admnum         0
fltno          1
visatype       0
dtype: int64

In [8]:
# Check for duplicate rows
im_df.duplicated().sum()

0

In [9]:
# Checked valid max and min values using .describe() above

# Update dtypes as needed
im_df['cicid'] = im_df['cicid'].astype(object)
im_df['arrdate'] = pd.to_datetime(im_df['arrdate']).dt.date
im_df['depdate'] = pd.to_datetime(im_df['depdate']).dt.date
im_df['dtadfile'] = pd.to_datetime(im_df['dtadfile']).dt.date
im_df['i94yr']=im_df['i94yr'].astype(int)
im_df['i94mon']=im_df['i94mon'].astype(int)
im_df['i94cit']=im_df['i94cit'].astype(int)
im_df['i94res']=im_df['i94res'].astype(int)
#im_df['i94mode']=im_df['i94mode'].astype(int)
im_df['i94bir']=im_df['i94bir'].astype(int)
im_df['i94visa']=im_df['i94visa'].astype(int)
im_df['count']=im_df['count'].astype(int)
im_df['biryear']=im_df['biryear'].astype(int)

# Replace invalid values, then update remaining data type
im_test = np.where(im_df['dtaddto']=='D/S', '01011900', im_df['dtaddto'])
im_df['dtaddto'] = im_test
im_df['dtaddto'] = pd.to_datetime(im_df['dtaddto'], format = '%m%d%Y').dt.date

In [10]:
# Drop all blank columns
im_df = im_df.drop(columns = ['occup', 'insnum'])

im_df.rename(columns={'i94bir': 'age'}, inplace=True)

# Clean column names - added here because not all files may have the same formatting
cols = list(im_df.columns)
cols = [x.lower().replace(' ','_') for x in cols]

im_df.columns = cols

In [11]:
# Quick count checks
im_df.groupby(['i94port'])['i94port'].count().sort_values(ascending = False)

i94port
NEW    312
NYC    137
MIA    116
CHI     59
LOS     56
ATL     53
WAS     52
SFR     39
PHI     25
BOS     24
HOU     15
SEA     13
FTL     13
DET     12
CLT     10
TOR     10
DAL      9
LVG      7
NAS      6
ORL      6
SPM      5
TAM      5
DEN      3
NOL      2
POO      2
MAA      1
OGG      1
HHW      1
HAM      1
PIT      1
SLC      1
VCV      1
AUS      1
XXX      1
Name: i94port, dtype: int64

In [12]:
im_df.groupby(['i94addr'])['i94addr'].count().sort_values(ascending = False)

i94addr
NY    374
FL    191
CA     94
IL     38
DC     33
NJ     30
NV     28
TX     24
GA     18
MA     16
MI     16
PA     12
WA     11
MD     11
PR     10
LA      9
CT      8
TN      7
MO      5
MN      4
NH      4
CO      4
AL      3
VA      3
AZ      3
NC      2
OK      2
OR      2
WC      1
HI      1
IN      1
ME      1
NE      1
OH      1
RI      1
SC      1
UT      1
AK      1
Name: i94addr, dtype: int64

In [13]:
im_df.groupby(['i94visa'])['i94visa'].count().sort_values(ascending = False)

i94visa
2    862
1    131
3      7
Name: i94visa, dtype: int64

In [14]:
im_df.groupby(['visatype'])['visatype'].count().sort_values(ascending = False)

visatype
WT    821
WB    104
B2     41
B1     19
F1      7
I       5
E2      3
Name: visatype, dtype: int64

## City Data

Steps include:
1. Check null count by column
2. Check duplicated rows
3. Check data types and update as needed
4.  Check column names and standardize as needed (all lowercase with underscores instead of spaces)
5.  Drop any columns that are completely empty
6.  Check extremes, max, min, and make sure numbers make sense (do areas that have high counts in real life also have high counts in this dataset and vice versa, are averages plausible, etc.)

In [ ]:
# Repeat for cities data
city_df.isnull().sum()

In [ ]:
city_df.duplicated().sum()

In [ ]:
city_df.dtypes

In [ ]:
city_df.groupby(['State'])['Total Population'].sum().sort_values(ascending = False)

In [10]:
city_df['pop_test'] = city_df['Male Population'] + city_df['Female Population']

state_level_total = city_df.groupby(['State'])['Total Population'].sum()#.sort_values(ascending = False)
state_level_male = city_df.groupby(['State'])['Male Population'].sum()
state_level_female = city_df.groupby(['State'])['Female Population'].sum()

state_agg = state_level_female.to_frame().join(state_level_male.to_frame())
state_agg = state_agg.join(state_level_total.to_frame()).reset_index()

state_agg['M_F Population'] = state_agg['Female Population'] + state_agg['Male Population'].astype(int)
state_agg['Female Population'] = state_agg['Female Population'].astype(int)
state_agg['Male Population'] = state_agg['Male Population'].astype(int)

state_agg['Pop_Diff'] = state_agg['Total Population'] - state_agg['M_F Population']

state_agg[state_agg['Pop_Diff'] > 0]

,State,Female Population,Male Population,Total Population,M_F Population,Pop_Diff
9,Florida,5859794,5387030,11392004,11246824.0,145180.0


In [15]:
# Clean column names
cols = list(city_df.columns)
cols = [x.lower().replace(' ','_') for x in cols]

city_df.columns = cols

## Airport Data

Steps include:
1. Check null count by column
2. Check duplicated rows
3. Check data types and update as needed
4.  Check column names and standardize as needed (all lowercase with underscores instead of spaces)
5.  Drop any columns that are completely empty
6.  Check extremes, max, min, and make sure numbers make sense (do areas that have high counts in real life also have high counts in this dataset and vice versa, are averages plausible, etc.)
7.  Split coordinates into latitude and longitude columns to make it easier to validate individually against other datasets.

In [ ]:
# Repeat for airports
airport_df.isnull().sum()

In [ ]:
airport_df.duplicated().sum()

In [ ]:
airport_df.dtypes

In [ ]:
airport_df.head()

In [ ]:
airport_df.groupby(['type'])['type'].value_counts().sort_values(ascending = False)

In [16]:
airport_df['airline'] = airport_df['local_code'].str[2:]
airport_df[['latitude', 'longitude']] = airport_df['coordinates'].str.split(",", expand = True).rename(columns = {0: 'latitude', 1: 'longitude'})

In [17]:
airport_df = airport_df.drop(columns = ['coordinates', 'continent', 'iata_code'])

## Date Dimension Table

### Create dataframe to be used for date and time dimension table by:
1. Create master date column to match date fields on
2. Create components from each date to be stored as separate columns

In [18]:
# adapted from here: https://stackoverflow.com/questions/34898525/generate-list-of-months-between-interval-in-python

dates = ["1900-01-01", "2020-01-01"]

def get_all_dates(dates):
    # format dates provided, assign them start and end signifiers
    start, end = [datetime.strptime(_, "%Y-%m-%d") for _ in dates]
    # return ordered dictionary of all days in between the start and end dates
    return OrderedDict(((start + timedelta(_)) \
                        .strftime(r"%Y-%m-%d"), None) for _ in range((end - start).days)).keys()

dates_all = get_all_dates(dates)#.rename(columns = '')

dates_all = pd.DataFrame(list(dates_all)).rename(columns = {0: 'dates_master'})
dates_all['dates_master'] = pd.to_datetime(dates_all['dates_master'])
dates_all['month'] = dates_all['dates_master'].dt.month
dates_all['day'] = dates_all['dates_master'].dt.day
dates_all['year'] = dates_all['dates_master'].dt.year
dates_all['weekday'] = dates_all['dates_master'].dt.weekday
dates_all['week'] = dates_all['dates_master'].dt.week
dates_all['dates_master'] = (dates_all['dates_master'].dt.date).astype(object)

In [ ]:
# Test joins
test_df = pd.merge(im_df, dates_all, left_on = 'dtaddto', right_on = 'dates_master')
test_df.head()

In [ ]:
test_df = pd.merge(im_df, airport_df, left_on = 'airline', right_on = 'airline')
#test_df[test_df['airline'] != 'None'].head()

test_df.groupby(['airline'])['airline'].value_counts().sort_values(ascending = False)

In [ ]:
test_df = pd.merge(city_df, airport_df, how='inner', left_on='city', right_on='municipality')
#test_df.head()
#test_df.groupby(['city'])['municipality'].value_counts().sort_values(ascending = False)

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

My goal of the data model is to create a STAR schema database for these files and the corresponding pipeline needed to import, clean, and create the tables.  My assumption is the data of primary interest relates to those immigrating on I9 visas.  Accordingly, I designed the below structure around this assumption.

### Fact Table
- immigrant I9 data

### Dimension Tables
- master dates table
- airport table
- city table
- state table

In [19]:
# Create the conceptual fact table schema in Spark
im_Schema = StructType([StructField("cicid", StringType(), True), \
                        StructField("i94yr", IntegerType(), True), \
                        StructField("i94mon", IntegerType(), True), \
                        StructField("i94cit", IntegerType(), True), \
                        StructField("i94res", StringType(), True), \
                        StructField("i94port", StringType(), True), \
                        StructField("arrdate", DateType(), True), \
                        StructField("i94mode", StringType(), True), \
                        StructField("i94addr", StringType(), True), \
                        StructField("depdate", StringType(), True), \
                        StructField("i94bir", IntegerType(), True), \
                        StructField("i94visa", IntegerType(), True), \
                        StructField("count", IntegerType(), True), \
                        StructField("dtadfile", StringType(), True), \
                        StructField("visapost", StringType(), True), \
                        StructField("entdepa", StringType(), True), \
                        StructField("entdepd", StringType(), True), \
                        StructField("entdepu", StringType(), True), \
                        StructField("matflag", StringType(), True), \
                        StructField("biryear", IntegerType(), True), \
                        StructField("dtaddto", DateType(), True), \
                        StructField("gender", StringType(), True), \
                        StructField("airline", StringType(), True), \
                        StructField("admnum", DoubleType(), True), \
                        StructField("fltno", StringType(), True), \
                        StructField("visatype", StringType(), True)])

spark_im_df = spark.createDataFrame(im_df,schema=im_Schema)
#spark_im_df.write.mode("append").parquet("/results/fact_immigration.parquet")

In [41]:
# Create the conceptual date dimension table
dates_Schema = StructType([StructField("dates_master", StringType(), True), \
                        StructField("month", IntegerType(), True), \
                        StructField("day", IntegerType(), True), \
                        StructField("year", IntegerType(), True), \
                        StructField("weekday", StringType(), True), \
                        StructField("week", StringType(), True)])

spark_dates_df = spark.createDataFrame(dates_all,schema=dates_Schema)

#write table
spark_dates_df.write.mode("append").parquet("/results/dim_dates.parquet")

In [42]:
# Create the conceptual airlines dimension table
airport_Schema = StructType([StructField("ident", StringType(), True), \
                        StructField("type", StringType(), True), \
                        StructField("name", StringType(), True), \
                        StructField("elevation_ft", DoubleType(), True), \
                        StructField("iso_country", StringType(), True), \
                        StructField("iso_region", StringType(), True),\
                        StructField("municipality", StringType(), True), \
                        StructField("gps_code", StringType(), True), \
                        StructField("local_code", StringType(), True), \
                        StructField("latitude", StringType(), True), \
                        StructField("longitude", StringType(), True), \
                        StructField("airline", StringType(), True)])

spark_airport_df = spark.createDataFrame(airport_df,schema=airport_Schema)

spark_airport_df.write.mode("append").parquet("/results/dim_airport.parquet")

In [20]:
city_df.head()

,city,state,median_age,male_population,female_population,total_population,number_of_veterans,foreign-born,average_household_size,state_code,race,count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402


In [24]:
geo_Schema = StructType([StructField("city", StringType(), True), \
                        StructField("state", StringType(), True), \
                        StructField("median_age", DoubleType(), True), \
                        StructField("male_population", DoubleType(), True), \
                        StructField("female_population", DoubleType(), True), \
                        StructField("total_population", IntegerType(), True), \
                        StructField("number_of_veterans", DoubleType(), True), \
                        StructField("foreign_born", DoubleType(), True), \
                        StructField("average_household_size", DoubleType(), True), \
                        StructField("state_code", StringType(), True), \
                        StructField("race", StringType(), True), \
                        StructField("count", IntegerType(), True)])

spark_city_df = spark.createDataFrame(city_df,schema=geo_Schema)

spark_city_df.write.mode("append").parquet("/results/dim_geo.parquet")

# Reference Capstone_Modularized.ipynb for remaining steps